<a href="https://colab.research.google.com/github/1kaiser/Gradient-based-depth-map-fusion/blob/main/Depth_Map_LeRes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installs a conda environment and restarts the session

In [ ]:
!python -m pip install -q condacolab
import condacolab
condacolab.install()
exit() ## for restarting the session

clones the github repository >>> creates the conda environment >>> updates the environment with necessary libraryriies

In [ ]:
%cd /content/
!git clone https://github.com/1kaiser/Gradient-based-depth-map-fusion.git
%cd Gradient-based-depth-map-fusion
!conda create --name GBDF
!conda activate GBDF
!conda env update -f GBDF.yaml
!python -m pip install opencv-python==4.6.0.66 torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install tqdm tensorboard setuptools==59.5.0 matplotlib scipy pip install timm==0.4.12 mmcv==0.6.2


Download Fusion Model checkpoints <<<

In [ ]:
model_dir = '/content/Gradient-based-depth-map-fusion/models'
import os
os.makedirs(model_dir, exist_ok=True)
!wget https://github.com/1kaiser/Gradient-based-depth-map-fusion/releases/download/v0.1/model_dict.pt -O {model_dir}'/model_dict.pt'

In [ ]:
#@title Resnet Model >>> segmentation model selection Box { vertical-output: true, display-mode: "form" }
model_selection = "res50.pth" #@param ["res50.pth", "res101.pth"] {allow-input: true}
yolov5_model_path = 'LeRes/'+str(model_selection)
!wget https://github.com/1kaiser/Gradient-based-depth-map-fusion/releases/download/v0.1/{model_selection} -O {yolov5_model_path}


**running the python Script 📜 on input files in a folder**

In [ ]:
!python run.py \
-p LeRes50 \
-i '/content/Gradient-based-depth-map-fusion/input' \
-o '/content/nbv' \
-m '/content/Gradient-based-depth-map-fusion/models/model_dict.pt'